In [1]:
import polars as pl 

In [20]:
hifi = pl.read_csv("data_hifi_pre_release.index.csv")
hifi_bams = (
    hifi.filter(pl.col("filetype").eq("bam") & pl.col("MM_remove").eq(False))
    #.select(["sample_ID", "filename", "path", "deepconsensus_path", "coverage", "deepconsensus_coverage", "basecaller", "library_ID", "instrument_model"])

)
hifi_bams.filter(pl.col("coverage") >= pl.col("coverage").max().over("sample_ID")).sort("coverage")

sample_ID,filetype,filename,path,data_type,production,coverage,deepconsensus_coverage,deepconsensus_filename,deepconsensus_path,ccs_algorithm,DeepConsensus_version,study,title,accession,bioproject_accession,biosample_accession,design_description,notes,generator_contact,generator_facility,library_ID,library_strategy,library_source,library_selection,library_layout,shear_method,size_selection,platform,instrument_model,seq_plate_chemistry_version,polymerase_version,total_reads,total_bp,total_Gbp,mean,min,max,N25,N50,N75,quartile_25,quartile_50,quartile_75,ntsm_score,MM_tag,primrose_filename,MM_review,MM_remove,lima_version,lima_float_version
str,str,str,str,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f64,i64,f64,f64,i64,i64,i64,i64,i64,i64,f64,bool,str,bool,bool,str,f64
"""HG00323""","""bam""","""m84081_230624_213331_s3.hifi_r…","""s3://human-pangenomics/working…","""unaligned reads""","""WUSTL_HPRC_HiFi_Year3_TopUp""",4.7,null,null,null,"""7.0.0""",null,null,null,null,"""PRJNA701308""",null,"""HiFi sequencing of 20kb fracti…",null,"""tgraves@wustl.edu""","""Washington University""","""HG00323_lib1""","""WGS""","""GENOMIC""","""size fractionation""","""single""","""Megaruptor 1""","""SageELF""","""PACBIO_SMRT""","""Revio""","""Revio C1""","""Revio P1""",766159,14435835846,14.44,18841,208.0,58203.0,16491,18872,21990,16069,18060,21073,0.635922,true,null,true,false,"""2.7.1""",2.0701
"""HG02273""","""bam""","""m84081_230630_193909_s2.hifi_r…","""s3://human-pangenomics/working…","""unaligned reads""","""WUSTL_HPRC_HiFi_Year3_TopUp""",6.7,null,null,null,"""7.0.0""",null,null,null,null,"""PRJNA701308""",null,"""HiFi sequencing of 20kb fracti…",null,"""tgraves@wustl.edu""","""Washington University""","""HG02273_lib1""","""WGS""","""GENOMIC""","""size fractionation""","""single""","""Megaruptor 1""","""SageELF""","""PACBIO_SMRT""","""Revio""","""Revio C1""","""Revio P1""",1032587,20811099072,20.81,20154,484.0,58922.0,17370,20326,23580,17011,19136,22531,0.669149,true,null,true,false,"""2.7.1""",2.0701
"""HG03195""","""bam""","""m84081_230628_190945_s2.hifi_r…","""s3://human-pangenomics/working…","""unaligned reads""","""WUSTL_HPRC_HiFi_Year3_TopUp""",7.0,null,null,null,"""7.0.0""",null,null,null,null,"""PRJNA701308""",null,"""HiFi sequencing of 20kb fracti…",null,"""tgraves@wustl.edu""","""Washington University""","""HG03195_lib1""","""WGS""","""GENOMIC""","""size fractionation""","""single""","""Megaruptor 1""","""SageELF""","""PACBIO_SMRT""","""Revio""","""Revio C1""","""Revio P1""",1155385,21595080464,21.6,18690,308.0,52888.0,16989,18248,20150,16796,17961,19554,0.613845,true,null,true,false,"""2.7.1""",2.0701
"""HG02486""","""bam""","""m54329U_200315_215927.ccs.bam""","""s3://human-pangenomics/working…","""unaligned reads""","""UW_HPRC_HiFi_Y1""",8.9,10.38,"""HG02486.m54329U_200315_215927.…","""s3://human-pangenomics/submiss…","""ccs 4.0.0 (commit SL-release-8…",null,"""SRP305758""",null,"""SRR13684277""","""PRJNA701308""","""SAMN17861238""","""HiFi sequencing of 18kb fracti…",null,"""kmiyamot@uw.edu""","""University of Washington""","""HG02486.HiFiEx2_f2""","""WGS""","""GENOMIC""","""size fractionation""","""single""","""g-TUBE""","""SageELF""","""PACBIO_SMRT""","""Sequel II""","""C2.0""","""P2.0""",1511142,27656723390,27.66,18301,48.0,50334.0,16803,18321,20131,16539,17995,19722,null,false,null,false,false,null,null
"""HG01928""","""bam""","""m64043_200612_200936.ccs.bam""","""s3://human-pangenomics/working…",null,"""WUSTL_HPRC_HiFi_Year1""",9.2,10.85,"""HG01928.m64043_200612_200936.d…","""s3://human-pangenomics/submiss…","""8.0.0.80529""",null,"""SRP305758""",null,"""SRR13684376""","""PRJNA701308""","""SAMN17861660""","""HiFi sequencing of 20kb fracti…",null,"""tgraves@wustl.edu""","""Washington University""","""HG01928_lib1""","""WGS""","""GENOMIC""","""size fractionation""","""single""","""Megaruptor 1""","""SageELF""","""PACBIO_SMRT""","""Sequel II""",null,